### This exercise aims to train a model to classify amazon product reviews as positive or negative

### __label1__ : negative
### __label2__ : positive

In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/romane/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/romane/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/romane/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/romane/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/romane/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/romane/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading packag

True

In [2]:
# load and inspect the data #amazon-reviews.csv
data = pd.read_csv('amazon-reviews.csv',encoding='latin_1')
data

,text,label
0,Stuning even for the non-gamer: This sound tr...,__label__2
1,The best soundtrack ever to anything.: I'm re...,__label__2
2,Amazing!: This soundtrack is my favorite musi...,__label__2
3,Excellent Soundtrack: I truly like this sound...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After H...",__label__2
...,...,...
9995,A revelation of life in small town America in...,__label__2
9996,Great biography of a very interesting journal...,__label__2
9997,Interesting Subject; Poor Presentation: You'd...,__label__1
9998,Don't buy: The box looked used and it is obvi...,__label__1


In [3]:
data['text'] = [word_tokenize(entry.lower()) for entry in data['text'].dropna()]

In [4]:
#defaultdict is a dictionary that provides a default value if the index is not found
#in tis example, the dictionnary efaults to nouns

tag_map = defaultdict(lambda:wn.NOUN)
tag_map['J']= wn.ADJ
tag_map['V']= wn.VERB
tag_map['R']= wn.ADV

stopWords = stopwords.words('english')

word_Lemmatized = WordNetLemmatizer()

for index,entry in enumerate(data['text']):
    final_words = []
    
    for word, tag in pos_tag(entry):
        #below condition is to check for top words and consider only alphabets
        if word not in stopWords and word.isalpha():
            final_words.append(word_Lemmatized.lemmatize(word,tag_map[tag[0]]))
    #The fina processed set of WORD FOR EACH ITERATION WILL be stored in 'text_final'
    data.loc[index,'text_final']= str(final_words)

In [5]:
# split your data into training and testing sets
train_x, test_x, train_y, test_y = model_selection.train_test_split(data['text_final'], data['label'], test_size=0.3)

In [6]:
#label encoding
encoder = LabelEncoder()
train_y_encoded = encoder.fit_transform(train_y)
test_y_encoded = encoder.fit_transform(test_y)

In [7]:
# perform tfidf vectorization on your data
tfidf_vect = TfidfVectorizer(max_features=5000)
tfidf_vect.fit(data['text_final'])
train_x_tfidf = tfidf_vect.transform(train_x)
test_x_tfidf = tfidf_vect.transform(test_x)

In [8]:
# fit the training dataset on the Naive Bayes classifier
naive = naive_bayes.MultinomialNB()
naive.fit(train_x_tfidf, train_y_encoded)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [9]:
# predict the labels on validation dataset
predictions_NB = naive.predict(test_x_tfidf)


In [10]:
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score: ",accuracy_score(predictions_NB, test_y_encoded)*100)

Naive Bayes Accuracy Score:  84.23333333333333


In [11]:
# Now we should be able to provide input sentences for the model to classify
sentence_to_classify = "This product is good"
naive.predict(tfidf_vect.transform([sentence_to_classify]))

array([0])